In [15]:
print('VGG Net')

VGG Net


In [27]:
import numpy as np
import torch
from torch import nn
from torch.autograd import Variable
from torchvision.datasets import CIFAR10


In [ ]:
# 定义一个VGG的block
def vgg_block(num_convs, in_channels, out_channels):
    net = [nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
           nn.ReLU(True)]
    
    for i in range(num_convs-1):
        net.append(nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1))
        net.append(nn.ReLU(True))
        
    net.append(nn.MaxPool2d(2, 2))
    return nn.Sequential(*net)


    

In [29]:
block_demo = vgg_block(3, 64, 128)
print(block_demo)

Sequential(
  (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace)
  (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace)
  (4): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (5): ReLU(inplace)
  (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)


In [30]:
input_demo = Variable(torch.zeros(1, 64, 300, 300))
output_demo = block_demo(input_demo)
print(output_demo.shape)

torch.Size([1, 128, 150, 150])


In [31]:
# 定义一个函数对block做堆叠
def vgg_stack(num_convs, channels):
    net = []
    for n, c in zip(num_convs, channels):
        in_c = c[0]
        out_c = c[1]
        net.append(vgg_block(n, in_c, out_c))
        
    return nn.Sequential(*net)


In [32]:
vgg_net = vgg_stack((1,1,2,2,2), ((3,64), (64, 128), (128,256), (256, 512),(512,512)))
print(vgg_net)

Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (1): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (2): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (3): Sequential(
    (0): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=

In [33]:

test_x = Variable(torch.zeros(1, 3, 256, 256))
test_y = vgg_net(test_x)
print(test_y.shape)


torch.Size([1, 512, 8, 8])


In [34]:
class vgg(nn.Module):
    def __init__(self):
        super(vgg, self).__init__()
        self.feature = vgg_net
        self.fc = nn.Sequential(
            nn.Linear(512, 100),
            nn.ReLU(True),
            nn.Linear(100, 10)
        )
    
    def forward(self, x):
        x = self.feature(x)
        x = x.view(x.shape[0], -1)
        x = self.fc(x)
        return x


In [35]:
#  定义一个训练函数
import time

def train(net, train_data, test_data, epoch, optimizer, criterion):
    # 
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    if torch.cuda.device_count() > 1:
        print("Let's use", torch.cuda.device_count(), "GPUs!")
        # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
        net = nn.DataParallel(net)
    net.to(device)
    for i in range(epoch):
        train_loss = 0
        train_acc = 0
        valid_loss = 0
        valid_acc = 0
        start = time.time()
        
        # 训练阶段 
        net.train()
        for im, label in train_data:
            im = Variable(im)
            label = Variable(label)
            im = im.to(device)
            label = label.to(device)
            
            out = net(im)
            loss = criterion(out, label)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            _, pred = out.max(1)
            num_correct = (pred==label).sum().item()
            acc = num_correct/im.shape[0]
            train_acc += acc
        
        # 测试阶段
        net.eval()
        for im, label in test_data:
            im = Variable(im)
            label = Variable(label)
            im = im.to(device)
            label = label.to(device)
            
            out = net(im)
            loss = criterion(out, label)
            valid_loss += loss.item()
            _, pred = out.max(1)
            num_correct = (pred==label).sum().item()
            acc = num_correct/im.shape[0]
            valid_acc += acc
        
        end = time.time()
        time_use = end - start
        # 输出本轮结果
        print('Epoch {}, Train Loss: {:.6f}, Train Acc: {:.6f},\
         Valid Loss: {:.6f}, Valid Acc: {:.6f}, Time: {:.6f}, len(train_data):{}'
              .format(i, 
                      train_loss/len(train_data), 
                      train_acc/len(train_data), 
                      valid_loss/len(test_data), 
                      valid_acc/len(test_data), 
                      time_use,
                      len(train_data)))
        


In [36]:
# 准备数据
def data_tf(x):
    x = np.array(x, dtype='float32')/255
    x = (x-0.5)/0.5
    x = x.transpose((2, 0, 1)) # channel 放在第一维。
    x = torch.from_numpy(x)
    return x
train_set = CIFAR10('../data', train=True, transform=data_tf)
train_data = torch.utils.data.DataLoader(train_set, batch_size=32, shuffle=True)
test_set = CIFAR10('../data', train=False, transform=data_tf)
test_data = torch.utils.data.DataLoader(test_set, batch_size=32, shuffle=True)


In [37]:
# 开始训练
net = vgg()
optimizer = torch.optim.SGD(net.parameters(), lr=1e-1)
criterion = nn.CrossEntropyLoss()
train(net, train_data, test_data, 20, optimizer, criterion)



Epoch 0, Train Loss: 2.303468, Train Acc: 0.101368,         Valid Acc: 2.303345, Valid Acc: 0.099840, Time: 45.960831, len(train_data):1563
Epoch 1, Train Loss: 2.269176, Train Acc: 0.120022,         Valid Acc: 2.104997, Valid Acc: 0.236222, Time: 45.342551, len(train_data):1563
Epoch 2, Train Loss: 1.814172, Train Acc: 0.296985,         Valid Acc: 1.615363, Valid Acc: 0.382488, Time: 45.372320, len(train_data):1563
Epoch 3, Train Loss: 1.458140, Train Acc: 0.451516,         Valid Acc: 1.292550, Valid Acc: 0.530351, Time: 45.358127, len(train_data):1563
Epoch 4, Train Loss: 1.139639, Train Acc: 0.586852,         Valid Acc: 1.088528, Valid Acc: 0.605132, Time: 45.428266, len(train_data):1563
Epoch 5, Train Loss: 0.895896, Train Acc: 0.685141,         Valid Acc: 1.024470, Valid Acc: 0.642372, Time: 46.262565, len(train_data):1563
Epoch 6, Train Loss: 0.718106, Train Acc: 0.752499,         Valid Acc: 0.816562, Valid Acc: 0.719748, Time: 45.991889, len(train_data):1563
Epoch 7, Train Loss:

In [7]:
# orderDict
import collections
print("Regular dictionary")
d = {}
d['a'] = 'A'
d['b'] = 'B'
d['c'] = 'C'
for k ,v in d.items():
    print(k, v)
    

Regular dictionary
a A
b B
c C


In [8]:
print('Order dictionary')
d1 = collections.OrderedDict()
d1['a'] = 'A'
d1['b'] = 'B'
d1['c'] = 'C'
d1['2'] = '2'
d1['1'] = '1'
for k, v in d1.items():
    print(k, v)

Order dictionary
a A
b B
c C
2 2
1 1


In [14]:
# 单个星号代表这个位置接收任意多个非关键字参数，并转化成元表。
# 也就是b 会接受除了a之外的剩下的非关键字参数，需要注意的是加在形参面前代表的是收集参数，
# 如果*号加在了是实参上(例如第十四行)，代表的是将输入迭代器拆成一个个元素
def one(a, *b):
    print(b)

def two(*b):
    print(b)

c = [6, 7, 8, 9]
print('one(1,2,3,4,5,6):')
one(1,2,3,4,5,6)
print('one(*c):')
one(*c)
print('one(*d1):')
one(*d1)
print('one(c):')
one(c)
print('one(d1):')
one(d1)
print('two(c):')
two(c)
print('two(d1):')
two(d1)
print('two(*c):')
two(*c)
print('two(*d1):')
two(*d1)

one(1,2,3,4,5,6):
(2, 3, 4, 5, 6)
one(*c):
(7, 8, 9)
one(*d1):
('b', 'c', '2', '1')
one(c):
()
one(d1):
()
two(c):
([6, 7, 8, 9],)
two(d1):
(OrderedDict([('a', 'A'), ('b', 'B'), ('c', 'C'), ('2', '2'), ('1', '1')]),)
two(*c):
(6, 7, 8, 9)
two(*d1):
('a', 'b', 'c', '2', '1')
